In [1]:
import pandas as pd
import numpy as np
from glob import glob
import shutil


In [2]:
predict_maskrcnn = 'Submission/submission_5.2_2.csv'
predict_2_class = 'Submission/2-cls test pred.csv'

In [3]:
submission_maskrcnn = pd.read_csv(predict_maskrcnn)
submission_2_class= pd.read_csv(predict_2_class)

In [4]:
submission_maskrcnn

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.991 833.0 1180.0 1905.0 1650.0 11 0.7105 4...
1,004f33259ee4aef671c2b95d54e4be68,3 0.5402469038963318 977.29913 1407.1526 1940....
2,008bdde2af2462e86fd373a445d0f4cd,8 0.9046102166175842 1392.2184 830.75555 1741....
3,009bc039326338823ca3aa84381f17f1,7 0.2 1398.0 717.0 1764.0 1400.0 0 0.878452261...
4,00a2145de1886cb9eb88869c85d74080,7 0.8848969340324402 1210.9208 1244.5746 1815....
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,13 0.9601746201515198 2331.072 1222.5479 2416....
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.7134953654289244 1027.8 706.4 1424.2 1152....
2997,ffaa288c8abca300974f043b57d81521,7 0.3 1877.0 1254.0 2233.0 1779.0 10 0.9188280...
2998,ffc441e0c8b7153844047483a577e7c3,0 0.7017951962153117 997.6666666666666 642.333...


In [5]:
submission_2_class

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0.936325
2996,ff9fcc4087ed5e941209aa3fa948e364,0.963583
2997,ffaa288c8abca300974f043b57d81521,0.178720
2998,ffc441e0c8b7153844047483a577e7c3,0.225196


## Filter threshold

In [6]:
low_thr  = 0.05
high_thr = 0.95

In [7]:
pred = pd.merge(submission_maskrcnn, submission_2_class, on = 'image_id', how = 'left')
pred

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.991 833.0 1180.0 1905.0 1650.0 11 0.7105 4...,0.013326
1,004f33259ee4aef671c2b95d54e4be68,3 0.5402469038963318 977.29913 1407.1526 1940....,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,8 0.9046102166175842 1392.2184 830.75555 1741....,0.939700
3,009bc039326338823ca3aa84381f17f1,7 0.2 1398.0 717.0 1764.0 1400.0 0 0.878452261...,0.123799
4,00a2145de1886cb9eb88869c85d74080,7 0.8848969340324402 1210.9208 1244.5746 1815....,0.654006
...,...,...,...
2995,ff91fb82429a27521bbec8569b041f02,13 0.9601746201515198 2331.072 1222.5479 2416....,0.936325
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.7134953654289244 1027.8 706.4 1424.2 1152....,0.963583
2997,ffaa288c8abca300974f043b57d81521,7 0.3 1877.0 1254.0 2233.0 1779.0 10 0.9188280...,0.178720
2998,ffc441e0c8b7153844047483a577e7c3,0 0.7017951962153117 997.6666666666666 642.333...,0.225196


In [8]:
pred['PredictionString'].value_counts().iloc[[0]]

14 1.0 0.0 0.0 1.0 1.0    191
Name: PredictionString, dtype: int64

In [9]:
def filter_2cls(row, low_thr=low_thr, high_thr=high_thr):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [10]:
sub = pred.apply(filter_2cls, axis=1)
sub

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0.013326
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,8 0.9046102166175842 1392.2184 830.75555 1741....,0.939700
3,009bc039326338823ca3aa84381f17f1,7 0.2 1398.0 717.0 1764.0 1400.0 0 0.878452261...,0.123799
4,00a2145de1886cb9eb88869c85d74080,7 0.8848969340324402 1210.9208 1244.5746 1815....,0.654006
...,...,...,...
2995,ff91fb82429a27521bbec8569b041f02,13 0.9601746201515198 2331.072 1222.5479 2416....,0.936325
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.7134953654289244 1027.8 706.4 1424.2 1152....,0.963583
2997,ffaa288c8abca300974f043b57d81521,7 0.3 1877.0 1254.0 2233.0 1779.0 10 0.9188280...,0.178720
2998,ffc441e0c8b7153844047483a577e7c3,0 0.7017951962153117 997.6666666666666 642.333...,0.225196


In [11]:
sub['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1700
Name: PredictionString, dtype: int64

In [12]:
import os
os.getcwd()

'/home/daitran/Desktop/git/chest_x_ray_abnormalities_detection/MaskRCNN_implementation'

In [13]:
sub[['image_id', 'PredictionString']].to_csv('Submission/submission_5.2_3.csv',index = False)

In [59]:
final_df = pd.read_csv('Submission/submission_2.2_2.csv')
final_df

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.993 810 1286 1900 1584 11 0.948 622 307 84...
1,004f33259ee4aef671c2b95d54e4be68,3 0.998 988 1478 1853 1774 0 0.994 1292 609 15...
2,008bdde2af2462e86fd373a445d0f4cd,3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...
3,009bc039326338823ca3aa84381f17f1,3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...
4,00a2145de1886cb9eb88869c85d74080,3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,3 1.0 1242 1440 2382 1722 0 0.999 1626 660 187...
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.995 1107 708 1413 1035 3 0.92 774 1406 165...
2997,ffaa288c8abca300974f043b57d81521,11 0.984 1535 352 2117 589 0 0.92 1247 1479 18...
2998,ffc441e0c8b7153844047483a577e7c3,3 0.999 721 1318 1533 1591 11 0.89 1059 295 12...


In [60]:
final_df['PredictionString'][:30]

0     3 0.993 810 1286 1900 1584 11 0.948 622 307 84...
1     3 0.998 988 1478 1853 1774 0 0.994 1292 609 15...
2     3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...
3     3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...
4     3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...
5     0 0.994 1376 855 1664 1166 3 0.989 1224 1552 2...
6     11 0.991 681 474 1000 560 11 0.813 1199 531 14...
7     0 0.996 1181 692 1437 980 13 0.953 447 509 834...
8     3 1.0 1107 1456 2007 1743 0 0.986 1431 826 162...
9     11 0.953 1428 315 1720 426 9 0.804 0 853 301 1...
10    3 0.993 517 1158 1285 1366 0 0.992 856 488 100...
11    11 1.0 1152 408 1452 486 0 0.901 1722 1296 210...
12    6 0.854 1534 1419 1818 1843 14 0.8581095263361...
13    3 1.0 797 1261 1792 1560 0 0.995 1074 466 1404...
14    3 0.999 1201 1412 2232 1792 0 0.995 1529 755 1...
15    3 1.0 959 1349 2089 1622 0 0.999 1475 575 1759...
16    3 0.999 1230 1524 2184 1782 6 0.915 1866 588 2...
17    3 0.997 1338 1512 2268 1812 0 0.992 1698 7